In [1]:
from hops import hdfs
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.types import LongType
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1559755371379_0020,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.

In [13]:
    df = spark \
        .read \
        .format("parquet") \
        .load(hdfs.project_path() + "Resources/iot-benchmarks/data/topic-lwm2m-3303-temperature") \
        .filter(F.col('endpointClientName')=='node-latency-rem-1-1')

    df.cache().count()

1250

In [14]:
df.printSchema()

root
 |-- measurement: struct (nullable = true)
 |    |-- timestamp: long (nullable = false)
 |    |-- endpointClientName: string (nullable = false)
 |    |-- instanceId: integer (nullable = false)
 |    |-- gatewayName: string (nullable = false)
 |    |-- ipsoObject: struct (nullable = false)
 |    |    |-- sensorValue: double (nullable = false)
 |    |    |-- minMeasuredValue: double (nullable = true)
 |    |    |-- maxMeasuredValue: double (nullable = true)
 |    |    |-- minRangeValue: double (nullable = true)
 |    |    |-- maxRangeValue: double (nullable = true)
 |    |    |-- sensorUnits: string (nullable = true)
 |    |    |-- resetMinAndMaxMeasuredValues: boolean (nullable = true)
 |-- kafkaTimestamp: timestamp (nullable = true)
 |-- endpointClientName: string (nullable = true)

In [15]:
timeFmt = "yyyy-MM-dd HH:mm:ss.SSS"
sensDf = df \
    .select(F.to_timestamp(F.col('measurement.timestamp')/1000).alias('measurementTmp'), \
            'kafkaTimestamp', \
            (F.col('measurement.timestamp')).alias('measurementTmpLong'), \
            'endpointClientName') \
    .withColumn('kafkaTmpLong', F.substring_index('kafkaTimestamp', '.', -1).cast('float')) \
    .withColumn('kafkaTmpLong', F.when(F.col('kafkaTmpLong') < 100, F.col('kafkaTmpLong')*10).otherwise(F.col('kafkaTmpLong')).cast('long')) \
    .withColumn('kafkaTmpLong', (F.unix_timestamp('kafkaTimestamp', format=timeFmt)*1000 + F.col('kafkaTmpLong'))) \
    .withColumn("diffMilliseconds", (F.col('kafkaTmpLong') - F.col('measurementTmpLong')))
    

In [16]:
sensDf.show(5, False)

+-----------------------+-----------------------+------------------+--------------------+-------------+----------------+
|measurementTmp         |kafkaTimestamp         |measurementTmpLong|endpointClientName  |kafkaTmpLong |diffMilliseconds|
+-----------------------+-----------------------+------------------+--------------------+-------------+----------------+
|2019-06-12 15:37:57.708|2019-06-12 15:37:57.886|1560353877708     |node-latency-rem-1-1|1560353877886|178             |
|2019-06-12 15:37:59.711|2019-06-12 15:37:59.723|1560353879711     |node-latency-rem-1-1|1560353879723|12              |
|2019-06-12 15:38:01.708|2019-06-12 15:38:01.722|1560353881708     |node-latency-rem-1-1|1560353881722|14              |
|2019-06-12 15:38:03.711|2019-06-12 15:38:03.722|1560353883711     |node-latency-rem-1-1|1560353883722|11              |
|2019-06-12 15:38:05.708|2019-06-12 15:38:05.722|1560353885708     |node-latency-rem-1-1|1560353885722|14              |
+-----------------------+-------

In [17]:
sensDf.printSchema()

root
 |-- measurementTmp: timestamp (nullable = true)
 |-- kafkaTimestamp: timestamp (nullable = true)
 |-- measurementTmpLong: long (nullable = true)
 |-- endpointClientName: string (nullable = true)
 |-- kafkaTmpLong: long (nullable = true)
 |-- diffMilliseconds: long (nullable = true)

In [18]:
sensDf.agg(F.avg(F.col('diffMilliseconds'))).show()

+---------------------+
|avg(diffMilliseconds)|
+---------------------+
|               15.328|
+---------------------+

In [19]:
sensDf.sort(F.col('diffMilliseconds').desc()).show(10, False)

+-----------------------+-----------------------+------------------+--------------------+-------------+----------------+
|measurementTmp         |kafkaTimestamp         |measurementTmpLong|endpointClientName  |kafkaTmpLong |diffMilliseconds|
+-----------------------+-----------------------+------------------+--------------------+-------------+----------------+
|2019-06-12 15:37:55.716|2019-06-12 15:37:57.044|1560353875716     |node-latency-rem-1-1|1560353877440|1724            |
|2019-06-12 15:37:57.708|2019-06-12 15:37:57.886|1560353877708     |node-latency-rem-1-1|1560353877886|178             |
|2019-06-12 15:50:37.708|2019-06-12 15:50:37.731|1560354637708     |node-latency-rem-1-1|1560354637731|23              |
|2019-06-12 16:11:19.706|2019-06-12 16:11:19.728|1560355879706     |node-latency-rem-1-1|1560355879728|22              |
|2019-06-12 15:49:47.709|2019-06-12 15:49:47.73 |1560354587709     |node-latency-rem-1-1|1560354587730|21              |
|2019-06-12 16:09:23.706|2019-06

In [20]:
# sensDf = sensDf.where(F.col('diffMilliseconds') < 1000)
# sensDf.count()

In [21]:
%%spark -o sensDf

In [22]:
%%local
sensDf

/srv/hops/anaconda/anaconda/envs/python36/lib/python3.6/site-packages/autovizwidget/widget/utils.py:50: FutureWarning:

A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.



Output()